In [162]:
import requests
from pydantic import BaseModel
from typing import List
import pdb
import logging

In [163]:
logger = logging.getLogger('dev')
logger.setLevel(logging.INFO)

In [213]:
GRANT_TYPES = {
    "authorization_code": "authorization_code",
    "refresh_token": "refresh_token"
}
self_client_dict = {
    "code": "1000.0660b1c556f7d47be5d8ec5ae967fa38.4b056e389187c12d0694c0259f2a2dcf",
    "scope": ["Desk.tickets.ALL"],
    "expiry_time": 1610367630253,
    "client_id": "1000.QXWN1SUWKQC7F1F1D4C2R8BC00I3AE",
    "client_secret": "6a27c69dfb8e31e6a9ceea8ef5bf152fd41aea8c07",
    "grant_types": GRANT_TYPES,
    "redirect_uri": "https://patient.serenity.health",
}
zoho_urls = {
    "oauth": "https://accounts.zoho.com/oauth/v2/token",
    "base_url": "https://desk.zoho.com/api/v1",
}

# state=-5466400890088961855&code=1000.71f7dd93b5571315574f9b82315ea6d6.91844bdb4a86740d3dba8a7e5b655d40&location=us

In [165]:
type(self_client_dict.get("hi"))

NoneType

In [229]:
class ZohoSelfClient(BaseModel):
    scope: List[str]
    expiry_time: int = None
    client_id: str
    client_secret: str
    code: str
    grant_types: dict
    access_token: str = None
    refresh_token: str = None
    redirect_uri: str = "https://patient.serenity.health"
        
    def get_authorization_header(self) -> str:
        return f"Zoho-oauthtoken {self.access_token}"


In [195]:
# self_client = ZohoSelfClient(**self_client_dict)
# self_client.code = "1000.483de26728e1dc7d0eda05919d945d9f.f9610e4be5b43dedf46fa6fb104f7959"

In [227]:
def get_zoho_oauth_tokens(zoho_self_client: ZohoSelfClient, oauth_url: str = zoho_urls["oauth"])-> dict: 
    payload = {
        "code": zoho_self_client.code,
        "grant_type": zoho_self_client.grant_types["authorization_code"],
        "client_id": zoho_self_client.client_id,
        "client_secret": zoho_self_client.client_secret,
        "redirect_uri": zoho_self_client.redirect_uri,
    }
    response = requests.post(oauth_url, params=payload)
    try:
        if response.status_code == 200 and not response.json().get("error"):
            logging.info("Successfully retrieved oauth tokens")
            tokens = response.json()
            zoho_self_client.access_token = tokens["access_token"]
            zoho_self_client.refresh_token = tokens["refresh_token"]
            tokens["status"] = True
        else:
            logging.error("Failed to retrieve oauth tokens")
            tokens = dict()
            tokens["status"] = False
    except (ValueError, KeyError) as e:
        pass # there's no token
    return tokens

In [231]:
zoho_self_client_dict = zoho_self_client.dict()
zoho_self_client_dict

{'scope': ['Desk.tickets.ALL'],
 'expiry_time': 1610367630253,
 'client_id': '1000.QXWN1SUWKQC7F1F1D4C2R8BC00I3AE',
 'client_secret': '6a27c69dfb8e31e6a9ceea8ef5bf152fd41aea8c07',
 'code': '1000.483de26728e1dc7d0eda05919d945d9f.f9610e4be5b43dedf46fa6fb104f7959',
 'grant_types': {'authorization_code': 'authorization_code',
  'refresh_token': 'refresh_token'},
 'access_token': '1000.62364743ad3344f15fe51a8988eaef42.abd218be429b372f12bad73af6e5b8ac',
 'refresh_token': '1000.6b63c18345b5b986009b9eeeeaf3f73c.d9bbbdeb68b3fc081668bb97ef632ae9',
 'redirect_uri': 'https://patient.serenity.health'}

In [230]:
zoho_self_client = ZohoSelfClient(**self_client_dict)
zoho_self_client.get_authorization_header()

'Zoho-oauthtoken 1000.62364743ad3344f15fe51a8988eaef42.abd218be429b372f12bad73af6e5b8ac'

In [198]:
tokens = get_zoho_oauth_tokens(self_client)
self_client.dict()

{'scope': ['Desk.tickets.ALL'],
 'expiry_time': 1610367630253,
 'client_id': '1000.QXWN1SUWKQC7F1F1D4C2R8BC00I3AE',
 'client_secret': '6a27c69dfb8e31e6a9ceea8ef5bf152fd41aea8c07',
 'code': '1000.483de26728e1dc7d0eda05919d945d9f.f9610e4be5b43dedf46fa6fb104f7959',
 'grant_types': {'authorization_code': 'authorization_code',
  'refresh_token': 'refresh_token'},
 'access_token': '1000.97e3f45344a66886b10987c06ceac647.26257e8cc1e3320ffe7c94c784656c27',
 'refresh_token': '1000.6b63c18345b5b986009b9eeeeaf3f73c.d9bbbdeb68b3fc081668bb97ef632ae9',
 'redirect_uri': 'https://patient.serenity.health'}

In [228]:
def refresh_zoho_oauth_tokens(zoho_self_client: ZohoSelfClient, oauth_url: str = zoho_urls["oauth"])-> dict: 
    if not self_client.refresh_token:
        tokens = get_zoho_oauth_tokens(zoho_self_client)
        if not tokens["status"]:
            return tokens
    else:
        payload = {
            "refresh_token": zoho_self_client.refresh_token,
            "scope": zoho_self_client.scope,
            "grant_type": zoho_self_client.grant_types["refresh_token"],
            "client_id": zoho_self_client.client_id,
            "client_secret": zoho_self_client.client_secret,
            "redirect_uri": zoho_self_client.redirect_uri,
        }
        response = requests.post(oauth_url, params=payload)
        logging.warning(response.url)
        try:
            if response.status_code == 200 and not response.json().get("error"):
                tokens = response.json()
                zoho_self_client.access_token = tokens["access_token"]
                tokens["status"] = True
            else:
                tokens = dict()
                tokens["status"] = False
        except ValueError:
            pass # there's no token
        return tokens

# serenity_zoho_access_keys

In [232]:
tokens = refresh_zoho_oauth_tokens(zoho_self_client)
zoho_self_client.dict()

{'scope': ['Desk.tickets.ALL'],
 'expiry_time': 1610367630253,
 'client_id': '1000.QXWN1SUWKQC7F1F1D4C2R8BC00I3AE',
 'client_secret': '6a27c69dfb8e31e6a9ceea8ef5bf152fd41aea8c07',
 'code': '1000.483de26728e1dc7d0eda05919d945d9f.f9610e4be5b43dedf46fa6fb104f7959',
 'grant_types': {'authorization_code': 'authorization_code',
  'refresh_token': 'refresh_token'},
 'access_token': '1000.cff56660f3058f3d29a89c276ed5542c.9e231299133903556ff2a433b0f87438',
 'refresh_token': '1000.6b63c18345b5b986009b9eeeeaf3f73c.d9bbbdeb68b3fc081668bb97ef632ae9',
 'redirect_uri': 'https://patient.serenity.health'}

## Get all tickets

In [208]:
!curl -X GET https://desk.zoho.com/api/v1/tickets?include=contacts,assignee,departments,team,isRead\
  -H "Authorization:Zoho-oauthtoken 1000.62364743ad3344f15fe51a8988eaef42.abd218be429b372f12bad73af6e5b8ac"

{"data":[{"id":"286477000000086284","ticketNumber":"101","layoutId":"286477000000074011","email":"support@zohodesk.com","phone":"1 888 900 9646","subject":"Here's your first ticket.","status":"Closed","statusType":"Closed","createdTime":"2018-05-31T23:20:18.000Z","category":null,"language":null,"subCategory":null,"priority":null,"channel":"Email","dueDate":"2018-06-02T23:20:18.000Z","responseDueDate":null,"commentCount":"0","sentiment":null,"threadCount":"1","closedTime":"2021-01-11T14:30:02.000Z","onholdTime":null,"departmentId":"286477000000006907","contactId":"286477000000086242","productId":null,"assigneeId":"286477000000086033","teamId":null,"department":{"id":"286477000000006907","name":"clearspacelabs"},"contact":{"firstName":null,"lastName":"Lawrence","email":"support@zohodesk.com","mobile":null,"phone":"1 888 900 9646","type":null,"account":{"accountName":"Zoho","website":"https://www.zoho.com/","id":"286477000000086214"},"id":"286477000000086242"},"team":null,"assignee":{"id"

In [236]:
def get_all_tickets(zoho_self_client: ZohoSelfClient, params: dict = {}, base_url: str = zoho_urls["base_url"],) -> dict:    
    headers = {"Authorization": f"Zoho-oauthtoken {zoho_self_client.access_token}"}
    response = requests.get(f"{base_url}/tickets", headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
    else:
        data = {error: True}
    return data

In [264]:
def create_ticket(zoho_self_client: ZohoSelfClient, payload:dict, base_url: str = zoho_urls["base_url"],) -> dict:    
    headers = {"Authorization": f"Zoho-oauthtoken {zoho_self_client.access_token}"}
    response = requests.post(f"{base_url}/tickets", headers=headers, data=payload)
    if response.status_code == 201:
        data = response.json()
    else:
        data = {"error": True}
    return data, response

In [265]:
response = create_ticket(zoho_self_client, payload = ticket_data)

In [267]:
ticket_data = {
    "subject",
    "departmentId"
}

<Response [415]>

In [260]:
null = None

In [262]:
ticket_data = {
    "subCategory" : "Serenity Patient portal",
    "cf" : {
    "cf_permanentaddress" : null,
    "cf_dateofpurchase" : null,
    "cf_phone" : null,
    "cf_numberofitems" : null,
    "cf_url" : null,
    "cf_secondaryemail" : null,
    "cf_severitypercentage" : "0.0",
    "cf_modelname" : "F3 2017"
    },
    "productId" : "",
    "contactId" : "",
    "contact":{
        "email": "chris@clearspacelabs.com"
    },
    "subject" : "Test automated ticket creation",
    "dueDate" : "2021-01-21T16:16:16.000Z",
    "departmentId" : "1892000000006907",
    "channel" : "Email",
    "description" : "Description of automated",
    "language" : "English",
    "priority" : "High",
    "classification" : "",
    "assigneeId" : "",
    "phone" : "+250786745117",
    "category" : "general",
    "email" : "chris@clearspacelabs.com",
    "status" : "Open"
}

In [256]:
!curl -X GET https://desk.zoho.com/api/v1/tickets\
    -H "Authorization: Zoho-oauthtoken 1000.cff56660f3058f3d29a89c276ed5542c.9e231299133903556ff2a433b0f87438"\
    -d '{
            "subCategory" : "Serenity Patient portal",
            "cf" : {
            "cf_permanentaddress" : null,
            "cf_dateofpurchase" : null,
            "cf_phone" : null,
            "cf_numberofitems" : null,
            "cf_url" : null,
            "cf_secondaryemail" : null,
            "cf_severitypercentage" : "0.0",
            "cf_modelname" : "F3 2017"
            },
            "productId" : "",
            "contactId" : "",
            "contact":{
                "email": "chris@clearspacelabs.com"
            }
            "subject" : "Test automated ticket creation",
            "dueDate" : "2021-01-21T16:16:16.000Z",
            "departmentId" : "1892000000006907",
            "channel" : "Email",
            "description" : "Description of automated",
            "language" : "English",
            "priority" : "High",
            "classification" : "",
            "assigneeId" : "",
            "phone" : "+250786745117",
            "category" : "general",
            "email" : "chris@clearspacelabs.com",
            "status" : "Open"
        }'

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 31)